In [0]:
import pyspark.sql.functions as F

In [0]:
# 1. Leer la tabla silver
df = spark.table("dbdemos.default.colaboradores_consolidados")

# 2. Procesos de limpieza/consolidación con anonimización
df_gold = (
    df.withColumn("DocAfil", F.regexp_replace("DocAfil", "[^0-9]", ""))  # normaliza documento
      .withColumn("DocAfil_hash", F.sha2(F.col("DocAfil"), 256))         # anonimiza
      .drop("DocAfil")                                                   # elimina el original
      .withColumn("periodos_ordenados", F.array_distinct("periodos_ordenados"))
      .withColumn("periodos_ordenados", F.array_sort("periodos_ordenados"))
      .withColumn("n_periodos", F.size("periodos_ordenados"))
      .withColumn("porcentaje_laboral", 
                  F.when(F.col("porcentaje_laboral").isNull(), F.lit(0))
                   .when(F.col("porcentaje_laboral") > 100, F.lit(100))
                   .when(F.col("porcentaje_laboral") < 0, F.lit(0))
                   .otherwise(F.col("porcentaje_laboral")))
      .withColumn("promedio_salario", F.round(F.col("promedio_salario"), 2))
      .withColumn("es_continuo", 
                  F.when(F.col("max_periodo") - F.col("min_periodo") + 1 == F.col("n_periodos"), F.lit(True))
                   .otherwise(F.lit(False)))
      .withColumn("fecha_proceso", F.current_date())
)

# 3. Almacenamiento definitivo en Gold Layer
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("dbdemos.default.colaboradores")
)
